In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [149]:
df = pd.read_csv('ml2_train.csv')

In [150]:
df.head()

,Serial Number,Main Office,Branch Number,Established Date,Acquired Date,City,County,State,2010 Deposits,2011 Deposits,2012 Deposits,2013 Deposits,2014 Deposits,2015 Deposits,2016 Deposits
0,1,0,2866,02/06/1910,11/13/2004,Wales,Waukesha,WI,21386.0,23981.0,24825.0,26908.0,30681.0,30680.0,37564
1,2,0,2867,02/28/1910,11/13/2004,Germantown,Washington,WI,55454.0,56564.0,64732.0,73523.0,81357.0,89270.0,106177
2,3,0,2868,05/06/1907,11/13/2004,Brookfield,Waukesha,WI,90882.0,104300.0,125038.0,125906.0,132501.0,137363.0,145054
3,4,0,2869,06/20/1936,11/13/2004,Pewaukee,Waukesha,WI,45674.0,49288.0,53251.0,70402.0,74742.0,73837.0,85996
4,5,0,2870,04/22/1963,11/13/2004,Waukesha,Waukesha,WI,64181.0,72217.0,69920.0,80703.0,75971.0,73225.0,81706


In [151]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 15 columns):
Serial Number       3772 non-null int64
Main Office         3772 non-null int64
Branch Number       3772 non-null int64
Established Date    3772 non-null object
Acquired Date       2297 non-null object
City                3772 non-null object
County              3772 non-null object
State               3772 non-null object
2010 Deposits       3050 non-null float64
2011 Deposits       3212 non-null float64
2012 Deposits       3461 non-null float64
2013 Deposits       3615 non-null float64
2014 Deposits       3734 non-null float64
2015 Deposits       3771 non-null float64
2016 Deposits       3772 non-null int64
dtypes: float64(6), int64(4), object(5)
memory usage: 442.1+ KB


In [152]:
obj_col = ['State']
df = pd.get_dummies(df, columns=obj_col)

In [153]:
df.drop(['Serial Number','Main Office','Branch Number','County','City','Established Date','Acquired Date'],axis=1,inplace=True)

In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 32 columns):
2010 Deposits    3050 non-null float64
2011 Deposits    3212 non-null float64
2012 Deposits    3461 non-null float64
2013 Deposits    3615 non-null float64
2014 Deposits    3734 non-null float64
2015 Deposits    3771 non-null float64
2016 Deposits    3772 non-null int64
State_AZ         3772 non-null uint8
State_CA         3772 non-null uint8
State_CO         3772 non-null uint8
State_CT         3772 non-null uint8
State_DC         3772 non-null uint8
State_FL         3772 non-null uint8
State_GA         3772 non-null uint8
State_ID         3772 non-null uint8
State_IL         3772 non-null uint8
State_IN         3772 non-null uint8
State_KY         3772 non-null uint8
State_LA         3772 non-null uint8
State_MA         3772 non-null uint8
State_MI         3772 non-null uint8
State_NJ         3772 non-null uint8
State_NV         3772 non-null uint8
State_NY         3772 non-null

In [155]:
X=df.drop('2016 Deposits',axis=1)

In [156]:
X=X.fillna(0)

In [157]:
y= df['2016 Deposits']

In [168]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [192]:
datadm = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
xg_reg = xgb.XGBRegressor(objective ='reg:linear',colsample_bytree=0.3,
 learning_rate= 0.01,
 max_depth= 6,
 n_estimators= 99)

In [177]:
from sklearn.model_selection import GridSearchCV

In [189]:
params={
    'n_estimators':np.arange(0,100,1),
    'max_depth':np.arange(0,10,1),
    'learning_rate':[0.01],
    'colsample_bytree':np.arange(0.1,0.5,0.1)
}
xg_reg = xgb.XGBRegressor(objective ='reg:linear',eval_metric='mae')
gridser = GridSearchCV(xg_reg,params)
gridser.fit(X,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='mae', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': array([ 0,  1, ..., 98, 99]), 'max_depth': array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), 'learning_rate': [0.01], 'colsample_bytree': array([0.1, 0.2, 0.3, 0.4])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [190]:
gridser.best_params_

{'colsample_bytree': 0.30000000000000004,
 'learning_rate': 0.01,
 'max_depth': 6,
 'n_estimators': 99}

In [191]:
gridser.best_score_

0.3744979738518453

In [193]:
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)

In [194]:
preds

array([ 67085.234,  61294.117,  29965.107,  20716.014,  44374.668,
        33027.   ,  26510.969,  44821.023,  26618.809,  26049.6  ,
        65335.457,  72578.34 , 111235.53 ,  34645.87 ,  29965.107,
        17611.525, 139518.84 ,  82174.29 , 130438.71 ,  38861.777,
       133876.61 ,  21327.254, 107195.99 ,  29755.031,  40992.35 ,
        80907.3  ,  80736.69 ,  23172.172, 139448.64 ,  47653.957,
        20716.014,  95954.76 ,  39043.383,  21816.918,  31088.21 ,
        37839.453,  61322.79 ,  49367.426,  43622.637,  33657.87 ,
        33534.   ,  18712.432,  40995.92 ,  35998.445,  41559.633,
        28701.979,  44591.54 ,  60183.77 ,  66365.836,  48933.33 ,
        96552.22 ,  20716.014,  61833.47 ,  44480.34 ,  39532.527,
        82774.64 ,  47530.152,  36598.79 , 129931.06 ,  92003.35 ,
        21662.096,  39832.402,  62528.43 ,  21912.69 ,  39624.625,
        32148.7  ,  55066.797,  46324.047,  33195.957,  83188.27 ,
        79270.74 ,  39449.254, 182157.34 ,  27808.39 ,  65127.

In [195]:
y_test

967      83931
537      84912
3220     55516
3113     20466
1001     61519
43       37419
3517     40357
2888     70136
2860     45263
3346     47903
3043     95459
1426    100635
1208    172717
2445     53497
3285     47880
2876     18123
1905    222098
1522    129701
1226    219601
2349     49880
1563    246948
2652     11774
319     162568
3029     44086
637      62561
1586    104071
472     141270
2763     27595
1613    206075
1057     35899
         ...  
15       71102
2687     52671
75       80794
2883     38963
2237     58972
3038     37746
3112     45021
3456    104272
3150     60588
3241     27968
3079     21621
1317    191352
1411    147237
2297     33672
1309    104699
1445     60583
3493     37737
3060     75182
935      86794
2814     30056
735     196743
1630    226091
2537    167325
2634     49053
2082     35238
698      53812
235     127433
667      87717
1804    210360
315      36266
Name: 2016 Deposits, Length: 755, dtype: int64

In [196]:
from sklearn.metrics import mean_absolute_error
rmse = (mean_absolute_error(y_test, preds))
print("MAE: %f" % (rmse))

MAE: 35341.290700


In [197]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 145517.888801
